# Pandas Structures

Similar to NumPy, Pandas uses two data types: series (equivalent to vectors) and dataframes (equivalent to matrices). However, Pandas provides an additional layer of abstraction over the underlying array, including indices, column headers and database-style operations.

Notice the indices and data type declaration in the following series output.

## Series

In [ ]:
import pandas as pd
import numpy as np

data = pd.Series([0.25, 0.5, 0.75, 1.0])
data

A series (and a dataframe) can be decomposed to pull out its constituent elements. Series elements can be referenced similar to NumPy arrays. For dataframes, the translation from NumPy arrays is slightly different.

In [ ]:
data.values

In [ ]:
data.index

In [ ]:
data[1]

In [ ]:
data[1:3]

Basic and advanced data structures can be drawn into Pandas objects. Indices need not be numeric.

In [ ]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

In [ ]:
population['Texas']

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

## Dataframes

In [ ]:
# Let's define a new series for states that gives their areas
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)

# We can combine population and area data series for states into a dataframe
states = pd.DataFrame({'population': population,
                       'area': area})
states

In [ ]:
states.columns

Pandas automatically infers the data type for each column.

In [ ]:
states.population.dtype

There are many other ways to construct a dataframe. As we will see in subsequent tutorials, one common way is by reading in a csv file to a dataframe.

## Pandas indexing
There are several ways to index data via Pandas. It can sometimes be confusing when indices are numerical but not sequential. Pandas can help address this challenge.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

In [ ]:
# explicit index when indexing - return the element with index value 1
data[1]

In [ ]:
# implicit index when slicing - return the elements at index locations 1 and 2
data[1:3]

I find these results confusion. Fortunately, Pandas has a built-in wrapper that adds intuition for the user.

In [ ]:
data.loc[1:3] # Return the element with index values in the range 1 to 3 (inclusive). Can also provide a list of strings.

In [ ]:
data.iloc[1:3] # Return the element with index locations in the range 1 to 3 (exclusive of 3)

Dataframes can also reference columns using their names.

In [ ]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

In [ ]:
data['area']

In [ ]:
data.area

In [ ]:
data[['area','pop']]

We can filter for both indices and column names with dataframes.

In [ ]:
data.loc[:'Florida',:'pop'] # Return all rows from 'Florida' and above and all columns from 'pop' and left.

We can mix indices and column names using the ```.ix``` indexer.

In [ ]:
data.iloc[:3].loc[:,:'area'] # Return all rows with indices less than 3 (exclusive) and all colummns from 'area' and left (inclusive)

## Operating on data in Pandas

Pandas is built on top of NumPy, so the vector optimization we saw with NumPy is also available with Pandas.

### Arithmetic Operations

In [23]:
rng = np.random.RandomState(42) # uses the Mersenne Twister pseudo-random number generator
ser = pd.Series(rng.randint(0, 10, 4))

df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])

In [ ]:
np.exp(ser)

In [ ]:
np.sin(df * np.pi / 4)

In [ ]:
np.sin(df.A/df.B * np.pi / 4)

### Detecting missing data

A common data processing operation is indicating missing data. We can either indicate missing values using 1) a masking approach (e.g., a separate vector with ```1/0``` indicators) or 2) a sentinel value (e.g., ```-999```). The first approach adds overhead storage and computation requirements. Implementation of the second approach in Pandas depends on the data type. 

In [ ]:
vals1 = np.array([1, None, 3, 4]) # Using None is the Pythonic approach. It requires datatype object.
print(vals1.dtype)

vals2 = np.array([1, np.nan, 3, 4]) # Using None is the Pythonic approach. It requires datatype object.
print(vals2.dtype)

The datatype will affect the treatment of missing values when performing arithmetic operations on the data.

In [ ]:
vals1.sum() # produces a TypeError

In [ ]:
vals2.sum() # No error but gives nan

Pandas has two useful methods to detect null data: ```isnull()``` and ```notnull()```.

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

It can be useful to combine these methods with masking to filter out NA values from a Pandas dataframe or series. 

In [ ]:
data[data.notnull()]

### Dropping missing data

Pandas also contains two convenience functions to deal with NA values: ```dropna()``` removes NA values and ```fillna()``` fills NA values with a user-specified value. Dataframes have the additional functionality that we can specify whether to drop rows (default implementation) or columns containing NA.

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(axis=1)

There is an even finer specification available. Perhaps we are interested in keeping only rows (columns) with 3 or more non-NA values.

In [ ]:
df[3] = np.nan # add some more NA to our dataframe
df

In [ ]:
df.dropna(axis=0,thresh=3)

### Filling missing data

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

Fill NA values with a single value, say 0.

In [ ]:
data.fillna(0)

Fill NA values with the value from above in the column. 

In [ ]:
# forward-fill
data.fillna(method='ffill')

Fill NA values with the value from below in the column.

In [ ]:
# back-fill
data.fillna(method='bfill')

The same procedure as for Series, but we must specify an axis (default rows)

In [ ]:
df.fillna(method='bfill', axis=1) # backfill doesn't fill last column

In [ ]:
df.fillna(method='ffill', axis=1) # forward doesn't fill last row of first column because no non-NA value before it in row

## Hierarchical (multi-index) data
Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ```Series``` and ```DataFrame``` objects, respectively. Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys. It is common to make use of hierarchical indexing (also known as multi-indexing) to incorporate multiple index levels within a single index. In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ```Series``` and two-dimensional ```DataFrame``` objects.

Suppose we would like to track data about states from two different years. Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

With this indexing scheme, we can straightforwardly index or slice the series based on this multiple index.

In [ ]:
pop[('California', 2010):('Texas', 2000)] # note: this assumes we know the order that data appears in the series

Consider the case that we are interested in all data from 2010. How would be select these data?

In [ ]:
pop[[i for i in pop.index if i[1] == 2010]] # messy

Pandas provides a much clearer built-in method for handling multi-index data.

In [ ]:
index = pd.MultiIndex.from_tuples(index) # create a MultiIndex from the index tuple
pop = pop.reindex(index) # change the pop index to our new MultiIndex
pop

In [ ]:
pop[:, 2010] # Much simpler than the non-MultiIndex approach above

Pandas also recognizes that a ```MultiIndex``` could be used to create a ```DataFrame``` from a ```Series```.

In [ ]:
pop_df = pop.unstack()
pop_df

In [ ]:
pop_df.stack() # opposite operation to unstack()

```MultiIndex``` syntax becomes even more powerful when we move from a ```Series``` to a ```DataFrame```, which cannot be represented by creating a new dimension.

In [ ]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

We can combine the above functionalities with vector arithmetic to calculate the proportion of the population aged less than 18.

In [ ]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

In a ```DataFrame```, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.

In [ ]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit']) # uses cartesian product (all combinations of levels) to generate the MultiIndex
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type']) # uses cartesian product (all combinations of levels) to generate the MultiIndex

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

In [ ]:
health_data['Guido', 'HR'] # access heartrate data for Guido

We've previously seen that Pandas has built-in data aggregation methods, such as ```mean()```, ```sum()```, and ```max()```. For hierarchically indexed data, these can be passed a ```level``` parameter that controls which subset of the data the aggregate is computed on.

In [ ]:
data_mean = health_data.groupby(level='year').mean() # groupby() syntax can also by used with columns by passing the string name (without level=)
data_mean

## Combining datasets
One of the most powerful functionalities provided by Pandas is the ability to combine ```DataFrames``` and ```Series``` based on common column or index values.

For convenience, we'll define a function to generate ```DataFrames```.

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

In addition, we'll create a quick class that allows us to display multiple ```DataFrames``` side by side. The code makes use of the special ```_repr_html_``` method, which IPython (Jupyter) uses to implement its rich object display.

In [67]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

### Combining data using ```concatenate()```

The simplest approach is to use the ```concatenate()``` function, which works similar to the parallel function in NumPy.

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

Caution is warranted when using ```concatenate()``` because it can produce duplicate indices. Fortunately, Pandas has a built-in solution!

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

In [ ]:
try: # try to run a code snippet
    pd.concat([x, y], verify_integrity=True)
except ValueError as e: # if a value error is thrown, then run the below code
    print("ValueError:", e)

It is often the case that the ```DataFrames``` to be joined do not have common column names. In such cases, we must use the ```join``` option.

If not provided, ```concatenate()``` will default to using an outer join and fill missing values with ```NaN```.

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')") # Only include columns that are common between the DataFrames

### Combining data using ```merge()```
One essential feature offered by Pandas is its high-performance, in-memory join and merge operations. The main interface for this operation is the ```pd.merge``` function, and we'll see a few examples of how this can work in practice.

Perhaps the simplest type of merge expresion is the one-to-one join, which is in many ways very similar to column-wise ```concatenation()```. In the following example, ```pd.merge()``` recognizes that both ```DataFrames``` contain an 'employee' column. It will merge these ```DataFrames``` using this common column as an index.

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2', 'pd.merge(df1, df2)')

Often a ```DataFrame``` contains duplicate entries. This produces many-to-one joins. In the below example, there are multiple entries in ```df3``` for 'group' that match to 'supervisor' in ```df4```.

In [ ]:
df3 = pd.merge(df1, df2)
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

We can also have a many-to-many relationship. This will create duplicate rows in the resulting ```DataFrame```, one for each row with common row values between ```df1``` and ```df5```.

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

In the preceding example, ```merge()``` is using the default behavior of matching along all common columns between the ```DataFrames```. Often, we want to merge ```DataFrames``` based on a subset of columns/indices, which may not have matching names.

In [ ]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

In [ ]:
df6 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df6', 'pd.merge(df1, df6, left_on="employee", right_on="name")')

The above operation results in a duplicate column. We can drop this column using the ```drop()``` method.

In [ ]:
pd.merge(df1, df6, left_on="employee", right_on="name").drop('name', axis=1)

Sometimes, it is useful to merge ```DataFrames``` on an index.

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

### Specifying set arithmetic for joins

In all the preceding examples we have glossed over one important consideration in performing a join: the type of set arithmetic used in the join. This comes up when a value appears in one key column but not the other.

In [ ]:
df7 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df8 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df7', 'df8', 'pd.merge(df7, df8)')

Here we have merged two datasets that have only a single 'name' entry in common: Mary. By default, the result contains the intersection of the two sets of inputs; this is what is known as an inner join. We can specify this explicitly using the how keyword, which defaults to ```"inner"```.

In [ ]:
pd.merge(df6, df7, how='inner')

Other options for the how keyword are ```'outer'```, ```'left'```, and ```'right'```. An outer join returns a join over the union of the input columns, and fills in all missing values with NAs.

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

The left join and right join return joins over the left entries and right entries, respectively. 

## Working with Time Series Data
Pandas was developed in the context of financial modeling, so as you might expect, it contains a fairly extensive set of tools for working with dates, times, and time-indexed data. Date and time data comes in a few flavors, which we will discuss here:
- *Time stamps* reference particular moments in time (e.g., July 4th, 2015 at 7:00am).
- *Time intervals* and *periods* reference a length of time between a particular beginning and end point; for example, the year 2015. Periods usually reference a special case of time intervals in which each interval is of uniform length and does not overlap (e.g., 24 hour-long periods comprising days).
- *Time deltas* or *durations* reference an exact length of time (e.g., a duration of 22.56 seconds).

In this section, we will introduce how to work with each of these types of date/time data in Pandas. This short section is by no means a complete guide to the time series tools available in Python or Pandas, but instead is intended as a broad overview of how you as a user should approach working with time series. We will start with a brief discussion of tools for dealing with dates and times in Python, before moving more specifically to a discussion of the tools provided by Pandas.

### Dates and Times in Python
The Python world has a number of available representations of dates, times, deltas, and timespans. While the time series tools provided by Pandas tend to be the most useful for data science applications, it is helpful to see their relationship to other packages used in Python.

#### Native Python dates and times: ```datetime``` and ```dateutil```
Python's basic objects for working with dates and times reside in the built-in ```datetime``` module. Along with the third-party ```dateutil``` module, you can use it to quickly perform a host of useful functionalities on dates and times. For example, you can manually build a date using the ```datetime``` type.

In [ ]:
from datetime import datetime
datetime(year=2015, month=7, day=4)

Or, using the ```dateutil``` module, you can parse dates from a variety of string formats.

In [ ]:
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

Once you have a datetime object, you can do operations like printing the day of the week. We'll use one of the standard string format codes for printing dates (```"%A"```), which you can read about in the [strftime section](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) of Python's [datetime documentation](https://docs.python.org/3/library/datetime.html). Documentation of other useful date utilities can be found in dateutil's [online documentation](http://labix.org/python-dateutil). A related package to be aware of is ```pytz```, which contains tools for working with the most difficult piece of time series data: time zones.

In [ ]:
date.strftime('%A')

The power of ```datetime``` and ```dateutil``` lie in their flexibility and easy syntax: you can use these objects and their built-in methods to easily perform nearly any operation you might be interested in. Where they break down is when you wish to work with large arrays of dates and times: just as lists of Python numerical variables are suboptimal compared to NumPy-style typed numerical arrays, lists of Python datetime objects are suboptimal compared to typed arrays of encoded dates.

#### Dates and times in Pandas
Pandas builds upon all the tools just discussed to provide a ```Timestamp``` object, which combines the ease-of-use of ```datetime``` and ```dateutil`` with the efficient storage and vectorized interface of ```numpy.datetime64```. From a group of these ```Timestamp``` objects, Pandas can construct a ```DatetimeIndex``` that can be used to index data in a ```Series``` or ```DataFrame```.

For example, we can use Pandas tools to repeat the demonstration from above. We can parse a flexibly formatted string date, and use format codes to output the day of the week.

In [ ]:
date = pd.to_datetime("4th of July, 2015")
date

In [ ]:
date.strftime('%A')

Additionally, we can do NumPy-style vectorized operations directly on this same object.

In [ ]:
date + pd.to_timedelta(np.arange(12), 'D') # create a daily time series from the initial date to 12 days later

#### Pandas time series: Indexing by time
Where the Pandas time series tools really become useful is when you begin to index data by timestamps. For example, we can construct a ```Series``` object that has time indexed data:

In [ ]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data['2014-07-04':'2015-07-04'] # select data for specific dates

In [ ]:
data['2015'] # select data for specific year

#### Pandas time series data structures
This section will introduce the fundamental Pandas data structures for working with time series data:
- For *time stamps*, Pandas provides the ```Timestamp``` type. As mentioned before, it is essentially a replacement for Python's native ```datetime```, but is based on the more efficient ```numpy.datetime64``` data type. The associated Index structure is ```DatetimeIndex```.
- For *time Periods*, Pandas provides the ```Period``` type. This encodes a fixed-frequency interval based on ```numpy.datetime64```. The associated index structure is ```PeriodIndex```.
- For *time deltas* or *durations*, Pandas provides the ```Timedelta``` type. ```Timedelta``` is a more efficient replacement for Python's native ```datetime.timedelta``` type, and is based on ```numpy.timedelta64```. The associated index structure is ```TimedeltaIndex```.

The most fundamental of these date/time objects are the ```Timestamp``` and ```DatetimeIndex``` objects. While these class objects can be invoked directly, it is more common to use the ```pd.to_datetime()``` function, which can parse a wide variety of formats. Passing a single date to ```pd.to_datetime()``` yields a ```Timestamp```; passing a series of dates by default yields a ```DatetimeIndex```.

In [ ]:
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

Any ```DatetimeIndex``` can be converted to a ```PeriodIndex``` with the ```to_period()``` function with the addition of a frequency code; here we'll use ```'D'``` to indicate daily frequency.

In [ ]:
dates.to_period('D')

A ```TimedeltaIndex``` is created, for example, when a date is subtracted from another.

In [ ]:
dates - dates[0]

#### Regular sequences
To make the creation of regular date sequences more convenient, Pandas offers a few functions for this purpose: ```pd.date_range()``` for timestamps, ```pd.period_range()``` for periods, and ```pd.timedelta_range()``` for time deltas. We've seen that Python's ```range()``` and NumPy's ```np.arange()``` turn a startpoint, endpoint, and optional stepsize into a sequence. Similarly, pd.date_range() accepts a start date, an end date, and an optional frequency code to create a regular sequence of dates. By default, the frequency is one day.

In [ ]:
pd.date_range('2015-07-03', '2015-07-10')

Similar functionalities exist for time date.

In [ ]:
pd.timedelta_range(0, periods=10, freq='H') # generate a time sequence with 10 entries at an hourly frequency